In [1]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4" # specify which GPU(s) to be used
bm.disable_gpu_memory_preallocation()
bm.set_platform('gpu')

/home/spirit/anaconda3/envs/cy_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
s =bm.array([0.5,1,1.5])
phi = bm.array([[0.3313,0.8148,0.4364],[0.8835,0.3621,0.2182],[0.3313,0.4527,0.8729]])
b = phi.T @ s
w = phi.T @ phi
w[bm.diag_indices_from(w)] = 0
print(w)
print(b)

Array(value=Array([[0.        , 0.7397555 , 0.62646115],
                   [0.7397555 , 0.        , 0.82969487],
                   [0.62646115, 0.82969487, 0.        ]]),
      dtype=float32)
Array(value=Array([1.5460999, 1.44855  , 1.74575  ]), dtype=float32)


In [18]:
lamb = 0.1
class IF_rk2(bp.NeuGroupNS):
    def __init__(self, size, b,  **kwargs):
        super(IF_rk2, self).__init__(size, **kwargs)

        self.V = bm.Variable(bm.zeros(size))
        self.cross = bm.Variable(bm.zeros(size))
        self.spike = bm.Variable(bm.zeros(size, dtype=bool))
        self.input = bm.Variable(bm.zeros(size))
        self.b     = b

    def update(self):
        dt = bp.share.load('dt')
        V = self.V + dt * (self.b + self.input - lamb)
        self.spike.value = V >= 1.
        dt_cross = bm.where(self.spike > 0., dt * (1 - self.V)/(V - self.V), 0.)
        self.cross.value = bm.where(self.spike >0., bm.exp(dt_cross -dt), 0.)
        # self.cross.value = dt_cross
        self.V.value = bm.where(self.spike > 0., 0. - (dt_cross - dt) * (b + self.input -lamb)  , V)
        self.input[:] = 0

In [19]:
class Spiking_LCA_rk2(bp.DynamicalSystemNS):
    def __init__(self, num, w, b, scale=1.0, method='rk2'):
        super(Spiking_LCA_rk2, self).__init__()
        
        # network size
        num_neuron = int(num * scale)

        # pars = dict(V_rest=0., V_th=1., V_reset=0., tau=1., tau_ref=0., method = method,
        #                                 V_initializer=bp.init.OneInit(value=0.))
        # self.N = bp.neurons.LIF(num_neuron, **pars)
        self.N   = IF_rk2(num_neuron,b)

        # synapses
        self.w_matrix = w   # excitatory synaptic weight (voltage)
        # w_matrix  = 0.
        self.N2N = bp.synapses.Exponential(pre=self.N,
                                            post=self.N,
                                            conn= bp.connect.All2All(),
                                            g_max= -self.w_matrix,
                                            tau=1.,
                                            output= bp.synouts.CUBA(),
                                            method=method,
                                            comp_method='dense')
        
    def update(self):
        t = bp.share.load('t')
        dt = bp.share.load('dt')
        self.N2N()
        self.N.input += bm.where(self.N.spike > 0, - self.w_matrix @ self.N.cross, 0.) 
        self.N()

In [34]:
net_rk2    = Spiking_LCA_rk2(3,w,b)
total_period = 1500
runner = bp.DSRunner(net_rk2,monitors=['N.spike','N.V','N.cross'], dt = 0.01)
runner.run(total_period)

Predict 150000 steps: :   0%|          | 0/150000 [00:00<?, ?it/s]

Predict 150000 steps: : 100%|██████████| 150000/150000 [00:11<00:00, 13118.41it/s]


In [44]:
size_num = runner.mon['N.spike'].shape[0]
spike_calculate = runner.mon['N.spike']
sum_along_columns = np.sum(spike_calculate, axis=0)/total_period
print(sum_along_columns)

[6.79333333e-01 6.66666667e-04 1.21733333e+00]


In [25]:
print(runner.mon['N.cross'])

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [33]:
matrix = runner.mon['N.cross']
nonzero_elements = matrix[matrix != 0]
print(nonzero_elements[20:50])
print(bm.exp(-0.01))

[0.99463385 0.99385405 0.9926323  0.99881685 0.99795747 0.9930292
 0.9980582  0.999314   0.99106765 0.9975513  0.9908724  0.99114645
 0.99859184 0.9913829  0.9938216  0.9977323  0.9914427  0.99264354
 0.99117297 0.9987424  0.9944738  0.9989461  0.99573576 0.99732715
 0.99035317 0.99924135 0.997777   0.9980435  0.9909119  0.9973382 ]
0.99004984


In [7]:
class BaseExpSyn(bp.SynConn):
  def __init__(self, pre, post, conn, g_max=1., delay=0., tau=8.0, E=0., method='exp_auto'):
    super(BaseExpSyn, self).__init__(pre=pre, post=post, conn=conn)

    # check whether the pre group has the needed attribute: "spike"
    self.check_pre_attrs('spike')

    # check whether the post group has the needed attribute: "input" and "V"
    self.check_post_attrs('input', 'V')

    # parameters
    self.E = E
    self.tau = tau
    self.delay = delay
    self.g_max = g_max

    # use "LengthDelay" to store the spikes of the pre-synaptic neuron group
    self.delay_step = int(delay/bm.get_dt())
    self.pre_spike = bm.LengthDelay(pre.spike, self.delay_step)

    # integral function
    self.integral = bp.odeint(lambda g, t: -g / self.tau, method=method)